# Installing Libraires

In [1]:
!pip install --quiet --upgrade pip langchain langchain-core langchain-community langchain-openai

# Loading ENV Varibales

In [2]:
import os
from dotenv import load_dotenv

_ = load_dotenv("/teamspace/studios/this_studio/LLM-Playlist/.env")

AZURE_OPENAI_API_ENDPOINT: str = os.environ.get('AZURE_OPENAI_API_ENDPOINT')
AZURE_OPENAI_API_KEY: str = os.environ.get('AZURE_OPENAI_API_KEY')
GPT35_MODEL_NAME: str = os.environ.get('MODEL_NAME')
OPENAI_API_TYPE: str = os.environ.get('OPENAI_API_TYPE')
DEPLOYMENT_NAME: str = os.environ.get('DEPLOYMENT_NAME')
AZURE_OPENAI_API_VERSION: str = os.environ.get('AZURE_OPENAI_API_VERSION')

# Loading Azure Chat Model

In [3]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    name=GPT35_MODEL_NAME,
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_API_ENDPOINT,
    azure_deployment=DEPLOYMENT_NAME,
    api_version=AZURE_OPENAI_API_VERSION, 
    openai_api_type=OPENAI_API_TYPE,
    temperature=0,
    top_p=0.1,
    )

# Trying Out LCEL!!!

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [5]:
prompt = ChatPromptTemplate.from_template("Hi, tell me a joke about {topic}")
model = llm
output_parser = StrOutputParser()

chain = prompt | model | output_parser

print(chain.invoke({"topic":"AI"}))

Sure, here's a joke about AI:

Why did the AI go on a diet?

Because it had too many bytes!


In [6]:
print(chain.invoke({"topic":"ice cream"}))

Sure, here's a joke for you:

Why did the ice cream go to therapy?

Because it had too many sprinkles of anxiety!


In [7]:
print(prompt.invoke({"topic":"ice cream"}))

messages=[HumanMessage(content='Hi, tell me a joke about ice cream', additional_kwargs={}, response_metadata={})]


In [8]:
print(model.invoke(prompt.invoke({"topic":"ice cream"})))

content="Sure, here's a joke for you:\n\nWhy did the ice cream go to therapy?\n\nBecause it had too many sprinkles of anxiety!" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 16, 'total_tokens': 44}, 'model_name': 'gpt-35-turbo-16k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}} id='run-67474f78-65f9-4905-8b66-875163e06212-0' usage_metadata={'input_tokens': 16, 'output_tokens': 28, 'total_tokens': 44}


In [9]:
print(model.invoke(prompt.invoke({"topic":"ice cream"})).content)

Sure, here's a joke for you:

Why did the ice cream go to therapy?

Because it had too many sprinkles of anxiety!


In [10]:
print(output_parser.invoke(model.invoke(prompt.invoke({"topic":"ice cream"}))))

Sure, here's a joke for you:

Why did the ice cream go to therapy?

Because it had too many sprinkles of anxiety!


# "|" operaor

In [11]:
2 | 3 > 1

True

In [12]:
from abc import ABC, abstractmethod

class CRunnable:

    def __init__(self):
        self.next = None

    @abstractmethod
    def process(self, data):
        pass

    def invoke(self, data):
        processed_data = self.process(data)
        if self.next is not None:
            return self.next.invoke(processed_data)
        return processed_data

    def __or__(Self, other):
        return CRunnableSequence(self, other)

class CRunnableSequence(CRunnable):
    
    def __init__(self, first, second):
        super().__init__()
        self.first = first
        self.second = second

    def process(self, data):
        return data

    def invoke(self, data):
        first_result = self.first.invoke(data)
        return self.second.invoke(first_result)



# Runnables from Langcahin

In [13]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

In [14]:
chain = RunnablePassthrough() | RunnablePassthrough() | RunnablePassthrough()
chain.invoke("hello")

'hello'

In [15]:
def lower_to_upper(input: str) -> str:
    output = input.upper()
    return output

In [16]:
chain = RunnablePassthrough() | RunnableLambda(lower_to_upper) | RunnablePassthrough()
chain.invoke("hello")

'HELLO'

In [17]:
chain = RunnableLambda(lower_to_upper)
chain.invoke("hello")

'HELLO'

In [18]:
RunnableLambda(func=lambda x: x.upper()).invoke("Hello")

'HELLO'

In [19]:
chain = RunnableParallel({"x": RunnablePassthrough(), "y": RunnablePassthrough()})

In [20]:
chain.invoke("hello")

{'x': 'hello', 'y': 'hello'}

In [21]:
chain.invoke({"input1": "hello", "input2":"goodbye"})

{'x': {'input1': 'hello', 'input2': 'goodbye'},
 'y': {'input1': 'hello', 'input2': 'goodbye'}}

In [22]:
chain.invoke("hello")

{'x': 'hello', 'y': 'hello'}

In [23]:
chain = RunnableParallel({"x": RunnablePassthrough(), "y": RunnableLambda(func= lambda x: x.upper())})

In [24]:
chain.invoke("hello")

{'x': 'hello', 'y': 'HELLO'}

In [25]:
chain = RunnableParallel({"x": RunnablePassthrough(), "y": RunnableLambda(func= lambda x: x['input2'].upper())})

In [26]:
chain.invoke({"input1": "hello", "input2":"goodbye"})

{'x': {'input1': 'hello', 'input2': 'goodbye'}, 'y': 'GOODBYE'}

# Nested chains

In [27]:
def find_keys_to_uppercase(input: dict):
    output = input.get("input1", "not found").upper()
    return output 

In [28]:
chain = RunnableParallel({"x": RunnablePassthrough() | RunnableLambda(find_keys_to_uppercase), "y": lambda z: z["input2"].title()})

In [29]:
chain.invoke({"input1": "hello", "input2":"goodbye"})

{'x': 'HELLO', 'y': 'Goodbye'}

In [30]:
chain.invoke({"input2": "hello", "input2":"goodbye"})

{'x': 'NOT FOUND', 'y': 'Goodbye'}

In [31]:
chain = RunnableParallel({"x": RunnablePassthrough()})

In [32]:
chain.invoke({"input":"hello", "input2":"goodbye"})

{'x': {'input': 'hello', 'input2': 'goodbye'}}

In [33]:
def assign_func(input):
    return 100

def multiply(input):
    return input * 10

In [34]:
chain = RunnableParallel({"x": RunnablePassthrough()})
result = chain.invoke({"input1":"hello", "input2":"goodbye"})
print(result)

{'x': {'input1': 'hello', 'input2': 'goodbye'}}


In [35]:
chain = RunnableParallel({"x": RunnablePassthrough()}).assign(extra=RunnableLambda(assign_func))
result = chain.invoke({"input1":"hello", "input2":"goodbye"})
print(result)

{'x': {'input1': 'hello', 'input2': 'goodbye'}, 'extra': 100}


# Combine Multiple Chains

In [36]:
def extractor(input):
    return input.get("extra", "not found")

def cupper(upper):
    return str(upper).upper()

new_chain = RunnableLambda(extractor) | RunnableLambda(cupper)

In [37]:
new_chain.invoke({"extra":"test"})

'TEST'

In [38]:
new_chain.invoke({"input1":"hello", "input2":"goodbye"})

'NOT FOUND'

In [39]:
chain.invoke({"input1":"hello", "input2":"goodbye"})

{'x': {'input1': 'hello', 'input2': 'goodbye'}, 'extra': 100}

In [40]:
final_chain = chain | new_chain
final_chain.invoke({"input1":"hello", "input2":"goodbye"})

'100'